# Brute-force solving altervative stable states for different kinds of resource supplies.

Before we use self-written optimization, here, we should try python packages for solving lieanr or non-linear systems.

## First consider $N_S = N_R$

In [1]:
from scipy.optimize import fsolve # type: ignore
from pathlib import Path
import torch
import numpy as np

datapath = Path('./data/').expanduser()
#tensorfile = {'C_span': Csample, 'G_span': Gsample, 'R_span': Rsample, 'S_span': Ssample}
loaded = torch.load(datapath/'AltSSsamples.pt')
C_span = loaded['C_span']
G_span = loaded['G_span']
R_span = loaded['R_span']
S_span = loaded['S_span']

See one case

In [2]:
C = C_span[11]
G = G_span[11]
R = R_span[11]
S = S_span[11]

delta = R @ G.transpose(0,1)
gamma = R * (S @ C)

In [3]:
Ns_sub = 1
Nr = 12
Ns = 12

sub_cmbins = torch.combinations(torch.tensor(range(Ns)), r=Ns_sub) # keep r lower than 5
num_cmbins = sub_cmbins.size(dim=0)

# for cmbins = range(num_cmbins)
cmbins = 0
self_spe_name = sub_cmbins[cmbins]
uniques, counts = torch.cat((self_spe_name, torch.tensor(range(Ns)))).unique(return_counts=True)
other_spe_name = uniques[counts == 1]
#spe_names = {self_spe_name, other_spe_name}

# for other in [0,1]: (we choose other = 1)
other = 1
if other == 1:
    spe_name = other_spe_name
    complement = self_spe_name
else:
    spe_name = self_spe_name
    complement = other_spe_name

In [4]:
G_sub = G[spe_name,:]
C_sub = C[spe_name,:]
S_sub = S[spe_name]

delta_sub = delta[spe_name]

In [5]:
def fixedpoint(x,*para):
    (G_sub, C_sub, delta_sub, gamma) = para # which should be .numpy!!!
    Ns_sub = G_sub.shape[0]
    '''
    x[:Ns_sub]: species, the left are resources.  
    '''
    y = np.zeros(x.shape)
    y[:Ns_sub] = x[Ns_sub:] @ G_sub.T - delta_sub
    y[Ns_sub:] = x[Ns_sub:] * (x[:Ns_sub] @ C_sub) - gamma
    return y

In [6]:
para = (G_sub.numpy(), C_sub.numpy(), delta_sub.numpy(), gamma.numpy())
init = torch.cat((S_sub, R), dim=0).numpy()
sol = torch.tensor(fsolve(fixedpoint, init, args=para)).type('torch.FloatTensor')

Test feasibility

In [7]:
if torch.all(sol > 0):
    print("feasible")

Test invadability

In [8]:
invade = (sol[len(spe_name):] @ G.transpose(0,1) - delta)[complement]
if torch.all(invade < 0.0):
    print("uninvadable")

# for MacArthur, we should consider if the resources can grow as well.

We can try diferent initial solving starting points.

In [9]:
sols = torch.tensor([])
# G, C, R, S, all the stuff here
for ic in range(10):
    init = np.random.rand(len(spe_name)+Nr)
    sol = torch.tensor(fsolve(fixedpoint, init, args=para)).type('torch.FloatTensor')
    if torch.all(sol > 0): # feasible
        invade = (sol[len(spe_name):] @ G.transpose(0,1) - delta)[complement]
        if torch.all(invade < 0.0): # uninvadable
            Jstar = torch.zeros(len(spe_name)+Nr,len(spe_name)+Nr)
            Jstar[:len(spe_name),len(spe_name):] = torch.diag(sol[:len(spe_name)]) @ G_sub
            Jstar[len(spe_name):,:len(spe_name)] = - torch.diag(sol[len(spe_name):]) @ C_sub.transpose(0,1)
            Jstar[len(spe_name):,len(spe_name):] = - torch.diag(C_sub.transpose(0,1) @ sol[:len(spe_name)])

            Eig_J = torch.linalg.eigvals(Jstar).real
            if len(Eig_J[Eig_J >= 1.0e-3]) == 0:
                # stable!
                sols = torch.cat((sols, sol.unsqueeze(0)))


PCA method

In [163]:
sols = torch.rand(3,12)
if sols.size() != torch.Size([0]):
    prod = sols @ sols.transpose(0,1)
    prod = torch.diag((torch.diag(prod)**(-1/2))) @ prod @ torch.diag((torch.diag(prod)**(-1/2)))
    Eig_PCA = torch.linalg.eigvals(prod).real
    num_AltSS = len(Eig_PCA[Eig_PCA >= 1.0e-3])

len(spe_name)*torch.ones(num_AltSS) # type: ignore

tensor([11., 11., 11.])

In [164]:
from FindAltSSfuncsv1 import FindAltSS
FindAltSS(G,C,S,R)

tensor([12.])

## New sampling set!

Let's get 48 different $\rho$, and for each $\rho$, we can sample 16 different ($C$, $G$, $R^*$, and $S^*$)

In [165]:
import math
Ns = 12
Nr = 12

num_rho = 48
rho_span = torch.zeros(num_rho)
rho_span[:int(num_rho/2)] = torch.linspace(0, .85, steps=int(num_rho/2))
rho_span[int(num_rho/2):] = torch.linspace(.87, 1, steps=int(num_rho/2))
num_test = 16


Csample = torch.zeros(num_rho,num_test,Ns,Nr)
Gsample = torch.zeros(num_rho,num_test,Ns,Nr)
Rsample = torch.zeros(num_rho,num_test,Nr)
Ssample = torch.zeros(num_rho,num_test,Ns)

for i in range(num_rho):
    rho = rho_span[i]
    for j in range(num_test):
        sample = torch.rand(Ns,2,Nr)
        L = torch.tensor([[1, 0],
                        [rho, math.sqrt(1-rho**2)]]) # Cholesky decomposition

        sample = torch.matmul(L,sample)

        G = sample[0:Ns,0]
        Gsample[i,j] = G

        C = sample[0:Ns,1] # C has not been pushed away

        C = C @ torch.diag(0.01+ 0.99*torch.rand(Nr))
        Csample[i,j] = C

        Sstar = 0.01 + 0.99*torch.rand(Ns)
        Rstar = 0.01 + 0.99*torch.rand(Nr)
        Ssample[i,j] = Sstar
        Rsample[i,j] = Rstar

'''
datapath = Path('./data/').expanduser()
tensorfile = {'C_span': Csample, 'G_span': Gsample, 'R_span': Rsample, 'S_span': Ssample}
torch.save(tensorfile, datapath/'AltSSsamplesv1.pt')
'''

In [14]:
datapath = Path('./data/').expanduser()
loaded = torch.load(datapath/'AltSSsamplesv1.pt')

C_span = loaded['C_span']
G_span = loaded['G_span']
R_span = loaded['R_span']
S_span = loaded['S_span']

# Linear supply
l_span = 0.1 + 0.9*torch.rand(R_span.size())
kappa_span = torch.zeros(R_span.size())

# logistic supply
g_span = 0.1 + 0.9*torch.rand(R_span.size())
K_span = torch.zeros(R_span.size())

for i in range(C_span.shape[0]):
    for j in range(C_span.shape[1]):
        C = C_span[i,j]
        #G = G_span[i,j]
        Ss = S_span[i,j]
        Rs = R_span[i,j]

        kappa_span[i,j] = (Rs * (Ss @ C))/l_span[i,j] + Rs
        K_span[i,j] = (Ss @ C)/g_span[i,j] + Rs


In [15]:
tensorfile = {'l_span': l_span, 'kappa_span': kappa_span, 
              'g_span': g_span, 'K_span': K_span}
torch.save(tensorfile, datapath/'AltSSsamplesv1LinLog.pt')

In [27]:
from FindAltSSfuncsv1 import FindAltSSLog
G = G_span[0,0]
C = C_span[0,0]
Ss = S_span[0,0]
Rs = R_span[0,0]

delta = Rs @ G.transpose(0,1)
g = g_span[0,0]
K = K_span[0,0]
FindAltSSLog(G,C,delta,g,K)

tensor([])